In [1]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j as nx #using nxneo4j
from py2neo import Graph, Node 
import pandas

In [2]:
cGraph = Graph("neo4j+s://101fd6b7.databases.neo4j.io", auth=('neo4j', "gB9F-fD2doYqInIcXR3DJZwnvvDWm-ZpgvOJ3BGCl54"))

In [3]:
recipeList = pandas.read_csv('../CSV/recipeList.csv')
listOfRecipes = pandas.read_csv('../CSV/listOfRecipes.csv')

In [88]:
listOfRecipes

,Unnamed: 0,Links,Recipe Name
0,0,https://www.sbs.com.au/food/recipes/maltese-ka...,Maltese kapunata ...
1,1,https://www.sbs.com.au/food/recipes/maltese-ba...,Maltese baked ricotta pie ...
2,2,https://www.sbs.com.au/food/recipes/maltese-st...,Maltese-style bruschetta (hobż biż-żejt) ...
3,3,https://www.sbs.com.au/food/recipes/maltese-ra...,Maltese rabbit stew (stuffat tal-fenek) ...
4,4,https://www.sbs.com.au/food/recipes/bread-oil-...,Bread with oil (hobż biż-żejt) ...
...,...,...,...
201,201,https://maltesecuisine.com/2020/01/13/pumpkin-...,Pumpkin Soup – Soppa tal-Qargha Hamra
202,202,https://maltesecuisine.com/2020/01/13/kannoli/,Kannoli
203,203,https://maltesecuisine.com/2020/01/13/christma...,Christmas Pudding
204,204,https://maltesecuisine.com/2020/01/13/pork-coo...,Pork Cooked in Milk


In [4]:
uniqueIng = []

for index, row in recipeList.iterrows():
    temp = [row[1].replace("_", " "), row[2]]
    if temp not in uniqueIng:
        uniqueIng.append(temp)
        

In [5]:
uniqueRecipe = []

for index, row in listOfRecipes.iterrows():
    temp = [row[2], row[1]]
    if temp not in uniqueRecipe:
        uniqueRecipe.append(temp)

In [6]:
print(len(uniqueIng))

277


In [7]:
for currIng, ingType in uniqueIng:
    currIng = currIng.replace("'", "-")
    addNode = "CREATE (n:Ingredient {name : '" + currIng + "', type: '" + str(ingType) + "'})"
        
    cGraph.query(addNode)

In [8]:
for recipe in uniqueRecipe:
    addNode = "CREATE (n:Recipe {name : '" + recipe[0].replace("'", "-") + "', link : '" + recipe[1] + "'})"
    cGraph.query(addNode)

In [10]:
ingPairWeights = pandas.read_csv('../CSV/ingPairWeights.csv')
ingPairWeights

,First Ingredient,Second Ingredient,Normalized Common Recipe Weight,Normalized Common Compound weight
0,eggplant,salt,0.0,-1.000000
1,eggplant,olive_oil,0.0,-1.000000
2,eggplant,onion,0.0,-1.000000
3,eggplant,garlic,0.0,-1.000000
4,eggplant,green_bell_pepper,0.0,-1.000000
...,...,...,...,...
38221,tangerine,yogurt,0.0,0.025806
38222,tangerine,rum,0.0,0.122581
38223,cranberry,yogurt,0.0,0.083871
38224,cranberry,rum,0.0,0.316129


In [11]:
for index, row in ingPairWeights.iterrows():
    row[0] = row[0].replace("'", "-")
    row[1] = row[1].replace("'", "-")

    exists = cGraph.query("MATCH ({name : '" + row[0] + "'})-[r]-({name : '" + row[1] + "'}) RETURN r")
    exists = exists.data()

    if len(exists) == 0 and (row[2] != 0 or row[3] != 0):
        addEdge = "MATCH (a:Ingredient), (b:Ingredient) WHERE a.name = '" + row[0] + "' AND b.name = '" + row[1] + "' CREATE (a)-[r:IngPair {comRecWeight:" + str(row[2]) + ", comCompWeight:" + str(row[3]) + "}]->(b) RETURN type(r)"
        cGraph.query(addEdge)

KeyboardInterrupt: 

In [ ]:
for index, row in recipeList.iterrows():
    row[0] = row[0].replace("'", "-")
    row[1] = row[1].replace("'", "-")

    exists = cGraph.query("MATCH ({name : '" + row[0] + "'})-[r]-({name : '" + row[1] + "'}) RETURN r")
    exists = exists.data()

    if len(exists) == 0:
        addEdge = "MATCH (a:Recipe), (b:Ingredient) WHERE a.name = '" + row[0] + "' AND b.name = '" + row[1] + "' CREATE (a)-[r:RecIng]->(b) RETURN type(r)"
        cGraph.query(addEdge)